# Creating the HTLC-Success Transaction

In this section, we'll build a Lightning channel HTLC-success transaction from scratch using Python. We'll walk through each part of the transaction — how it's constructed and signed. The process will be tested using Bitcoin Core in regtest mode.

## Setup

For this notebook, we'll use Bob's commitment transaction created in `workshop-solution-chapter3` which contains the accepted HTLC output that we'll spend with the HTLC-success transaction.

In [ ]:
%run "../chapter 3 - in-flight htlc commitment transaction/workshop-solution-chapter3.ipynb"

## HTLC-Success Transaction

The HTLC-success transaction is a second-level transaction that spends an accepted HTLC output from a commitment transaction. It allows Bob (who received the HTLC) to claim the funds by providing the payment preimage before the timeout period expires.

**Key differences from HTLC-timeout:**
- Spends an **accepted HTLC** (Bob's perspective) instead of offered HTLC
- Requires the **payment preimage** to unlock
- Uses scriptB (success path) instead of scriptA (timeout path)
- Must be claimed before CLTV timeout expires

### The Unsigned HTLC-Success Transaction

#### The Input

The input is the accepted HTLC output from Bob's commitment transaction (output index 2).

In [ ]:
# Get the commitment transaction details
# Bob's commitment transaction has the accepted HTLC output at index 2
commitment_txid = decoded['txid']
commitment_output_index = 2

print(f"Spending from Bob's commitment tx: {commitment_txid}")
print(f"Output index: {commitment_output_index} (accepted HTLC)")

# VERSION
# version '2' indicates that we may use relative timelocks (BIP68)
version = bytes.fromhex("0200 0000")

# MARKER (new to segwit)
marker = bytes.fromhex("00")

# FLAG (new to segwit)
flag = bytes.fromhex("01")

# INPUTS
# We have just 1 input
input_count = bytes.fromhex("01")

# Convert txid and index to bytes (little endian)
commitment_txid_bytes = bytes.fromhex(commitment_txid)[::-1]
commitment_index = commitment_output_index.to_bytes(4, byteorder="little", signed=False)

# For the unsigned transaction we use an empty scriptSig
scriptsig = bytes.fromhex("")

# Sequence: must be set to 1 for the CSV delay in scriptB
# scriptB requires: OP_1 OP_CHECKSEQUENCEVERIFY
sequence_htlc = bytes.fromhex("01000000")

inputs = (
    commitment_txid_bytes
    + commitment_index
    + varint_len(scriptsig)
    + scriptsig
    + sequence_htlc
)

#### The Output

The HTLC-success transaction has a single output that pays to Bob with a delay. This output is similar to the `to_local` output in commitment transactions: it's spendable by Bob after a delay, or immediately by Alice if she has the revocation key (in case this commitment gets revoked).

#### To Local Delayed Output

    +------+---------------+
    | OP_1 |       Q       |
    +------+---------------+
                   ^
                   |   +-------------------+
                    ---| P(revocation) + T |
                       +-------------------+
                                         ^
                                         |
                                   +-----------+        
                                   | T = t * G |
                                   +-----------+        
                                         ^
                                         |
     +---+   +-------------------------------------------------------+
     | t | = | TaggedHash ("Taptweak", P(revocation) || script_root) |
     +---+   +-------------------------------------------------------+
                                                             ^
                                                             |
                                                          +-----+
                                                          |  h  |
                                                          +-----+                   
                                                             ^                                                          
                                                             |                                                          
      +-----------------------------------------------------------+                                           
      | P(local_delayed) OP_CHECKSIG to_self_delay OP_CSV OP_DROP |                                           
      +-----------------------------------------------------------+

**Note:** If option_anchors applies (which is the case here), then the HTLC-success transactions are signed with the input and output having the same value. This means they have a zero fee and MUST be combined with other inputs to arrive at a reasonable fee.

In [ ]:
# Output count
output_count = bytes.fromhex("01")

# Output value (same as HTLC input value for zero-fee transaction)
output_value_sat = htlc_output_value_satoshis
output_value = output_value_sat.to_bytes(8, byteorder="little", signed=False)

# We already have bob_delayed_pubkey and alice_revocation_pubkey from chapter 3
# These are the keys used for Bob's outputs

# TODO: Create the script: P(local_delayed) OP_CHECKSIG to_self_delay OP_CSV OP_DROP
to_self_delay = 144
script_output = # TODO

# TODO: Compute output script_root
hash_input = # TODO
script_root = # TODO

# TODO: Compute the output Tagged Hash (using Alice's revocation key since Bob holds her commitment)
taptweak = # TODO
htlc_alice_revocation_pubkey_tweaked = alice_revocation_pubkey.tweak_add(taptweak)

# Compute scriptPubKey P2TR: OP_1 (0x51) + PUSH32 (0x20) + xonly(32B)
output_spk = bytes.fromhex("51") + varint_len(htlc_alice_revocation_pubkey_tweaked.get_bytes(bip340=True)) + htlc_alice_revocation_pubkey_tweaked.get_bytes(bip340=True)

outputs = (
    output_value
    + varint_len(output_spk)
    + output_spk
)

# Locktime: set to 0 (no CLTV requirement for success path)
locktime = bytes.fromhex("00000000")

unsigned_tx = (
    version
    + input_count
    + inputs
    + output_count
    + outputs
    + locktime
)

print("\nunsigned_tx:", unsigned_tx.hex())

# Decode the unsigned transaction to verify it looks correct
decoded_htlc_success = node.decoderawtransaction(unsigned_tx.hex())
print(json.dumps(decoded_htlc_success, indent=2, default=str))

## The sighash for script path spend

Unlike the commitment transaction which uses **key path spending**, the HTLC-success transaction uses **script path spending** through the Taproot tree. We need to spend using scriptB of the accepted HTLC output.

The sighash calculation for script path spending includes additional data about the script being executed.

In [ ]:
# We're spending the accepted HTLC output using scriptB (success path)
spending_script = scriptB

# TODO: Calculate the tapleaf hash for the script we're spending
hash_input_spending = # TODO
tapleaf_hash = # TODO
key_version = bytes.fromhex("00")  # reserved for future upgrades
codeseparator = bytes.fromhex("ffffffff")  

# SIGHASH for script path spend (BIP-341)
index_of_this_input = bytes.fromhex("0000 0000")
sighash_epoch = bytes.fromhex("00")
hash_type = bytes.fromhex("83")  # SIGHASH_SINGLE|SIGHASH_ANYONECANPAY (0x03 | 0x80)

# When SIGHASH_SINGLE (0x03) is set, sha_outputs is 32 bytes (BIP-341)
sha_outputs = sha256(outputs).digest()

# Data about this input, script path (no annex)
spend_type = bytes.fromhex("02")

# TODO: Common signature message extension (BIP-341)
# When SIGHASH_ANYONECANPAY is set, we include data about THIS specific input:
outpoint = # TODO
amount = # TODO
nSequence = # TODO

sig_msg = (
    sighash_epoch
    + hash_type
    + version
    + locktime
    + spend_type
    + outpoint          
    + amount
    + varint_len(bob_accepted_htlc_spk)
    + bob_accepted_htlc_spk
    + nSequence
    + sha_outputs
    + tapleaf_hash
    + key_version
    + codeseparator
)

print("\nSignature message:", sig_msg.hex())

tag_hash = sha256("TapSighash".encode()).digest()
sighash = sha256(tag_hash + tag_hash + sig_msg).digest()
print("\nSighash:", sighash.hex())

## Signing the sighash

For the HTLC-success transaction, we need signatures from both Bob's and Alice's HTLC keys. These are regular Schnorr signatures (not MuSig2 aggregated signatures), since the script requires two separate signature checks.

In [ ]:
# TODO: Derive Bob's HTLC private key (local for Bob)
bob_htlc_basepoint = # TODO
bob_htlc_privkey = # TODO

# TODO: Derive Alice's HTLC private key (remote for Bob)
alice_htlc_basepoint = # TODO
alice_htlc_privkey = # TODO

# Generate auxiliary random data for each signature
import secrets
aux_bob = secrets.token_bytes(32)
aux_alice = secrets.token_bytes(32)

# TODO: Sign with Bob's HTLC key (for OP_CHECKSIGVERIFY)
bob_htlc_sig = # TODO
# APPEND sighash type byte for non-default sighash!
bob_htlc_sig = bob_htlc_sig + hash_type  # Add 0x83 byte

# Verify Bob's signature (without the sighash byte for verification)
bob_sig_valid = bob_htlc_pubkey.verify_schnorr(bob_htlc_sig[:-1], sighash)
print(bob_htlc_sig.hex())
print("Bob signature valid?", bob_sig_valid)
print(f"Bob signature length: {len(bob_htlc_sig)} bytes (should be 65)")

# TODO: Sign with Alice's HTLC key (for OP_CHECKSIG)
alice_htlc_sig = # TODO
# APPEND sighash type byte for non-default sighash!
alice_htlc_sig = alice_htlc_sig + hash_type  # Add 0x83 byte

# Verify Alice's signature (without the sighash byte for verification)
alice_sig_valid = alice_htlc_pubkey.verify_schnorr(alice_htlc_sig[:-1], sighash)
print(alice_htlc_sig.hex())
print("Alice signature valid?", alice_sig_valid)
print(f"Alice signature length: {len(alice_htlc_sig)} bytes (should be 65)")

## The signed transaction

Now we construct the witness for the script path spend. The witness stack for spending scriptB (success path) contains:
1. Alice's signature (last on stack, for OP_CHECKSIG)
2. Bob's signature (for OP_CHECKSIGVERIFY)
3. The payment preimage (to satisfy the hash check)
4. The script being executed (scriptB)
5. The control block (proving scriptB is in the taproot tree)

In [ ]:
# TODO: Construct the control block
# Control block format: <version byte> <internal key> <parity bit> [<merkle proof>]
# Version byte: 0xc0 | (parity of Q)

# Get parity bit from Q
parity = # TODO
version_byte = # TODO

# TODO: The control block includes the merkle proof (taggedhash_leafA since we're spending leafB)
control_block = # TODO

print("\ncontrol_block:", control_block.hex())
print("\nspending_script:", spending_script.hex())
print("\npayment_preimage:", payment_preimage.hex())

# TODO: Construct witness stack
# Stack order (bottom to top): <preimage> <bob_sig> <alice_sig> <script> <control_block>
witness = # TODO

# The final signed transaction
signed_tx = (
    version
    + marker
    + flag
    + input_count
    + inputs
    + output_count
    + outputs
    + witness
    + locktime
)

print("\nsigned tx:", signed_tx.hex())

# Before testing it, we need to confirm the commitment transaction so that the HTLC output exists on-chain
bob_commitment_txid = node.sendrawtransaction(signed_bob_commitment_tx.hex())
result = node.generatetoaddress(nblocks=1, address=address, called_by_framework=True)

print("bob_commitment_txid: ", bob_commitment_txid)

# Now we can test if the HTLC timeout transaction would be accepted in the mempool (it shouldn't, since it is paying no fee)
print("\nTest mempool accept:")
try:
    result = node.testmempoolaccept(rawtxs=[signed_tx.hex()])
    print(json.dumps(result, indent=2, default=str))
except Exception as e:
    print(f"Expected error (commitment tx not in mempool): {e}")

## Adding input for fees

As stated in the BOLTs: "If option_anchors applies, then the HTLC-timeout and HTLC-success transactions are signed with the input and output having the same value. This means they have a zero fee and MUST be combined with other inputs to arrive at a reasonable fee."

We'll now add Bob's sweeper output created in chapter 2 as a second input to pay for fees.

In [ ]:
# INPUTS
# We have 2 inputs: the HTLC output and bob sweeper output
input_count_v2 = bytes.fromhex("02")

# INPUT 2: bob sweeper output from chapter 2
# TODO: Get Bob's sweeper transaction details from chapter 2
bob_txid_to_spend_bytes = # TODO
bob_sweeper_index_bytes = # TODO
sequence_change = bytes.fromhex("ffffffff")

inputs_v2 = (
    commitment_txid_bytes
    + commitment_index
    + varint_len(scriptsig)
    + scriptsig
    + sequence_htlc
    + bob_txid_to_spend_bytes
    + bob_sweeper_index_bytes
    + varint_len(scriptsig)
    + scriptsig
    + sequence_change
)

print(f"Added bob_sweeper input from funding tx: {funding_channel_txid}:{bob_sweeper_index}")

In [ ]:
# OUTPUTS
# Now we have 2 outputs: HTLC delayed output + change back to bob
output_count_v2 = bytes.fromhex("02")

# OUTPUT 1: HTLC delayed output (same as before)
# Reuse the same output_value and output_spk from v1

# OUTPUT 2: Change back to bob_change address
# TODO: Calculate: bob_sweeper_value (98,999,700) + htlc_value (500,000) - htlc_output (500,000) - fee
tx_fee_sat = 300
bob_change_value = # TODO
bob_change_value_bytes = bob_change_value.to_bytes(8, byteorder="little", signed=False)

# scriptPubKey P2TR: OP_1 (0x51) + PUSH32 (0x20) + bob_change_pubkey
output_spk_change = bytes.fromhex("51") + varint_len(bob_change_pubkey.get_bytes(bip340=True)) + bob_change_pubkey.get_bytes(bip340=True)

# Use outputs_v2 to avoid overwriting the original outputs variable
outputs_v2 = (
    output_value
    + varint_len(output_spk)
    + output_spk
    + bob_change_value_bytes
    + varint_len(output_spk_change)
    + output_spk_change
)

print(f"Transaction fee: {tx_fee_sat} sats")

In [ ]:
unsigned_tx_v2 = (
    version
    + input_count_v2
    + inputs_v2
    + output_count_v2
    + outputs_v2
    + locktime
)

print("unsigned_tx_v2:", unsigned_tx_v2.hex())

# Decode the unsigned transaction to verify it looks correct
decoded_htlc_success_v2 = node.decoderawtransaction(unsigned_tx_v2.hex())
print(json.dumps(decoded_htlc_success_v2, indent=2, default=str))

### Sign the bob sweeper input (Input 1 - key path spend)

The HTLC input signatures were already created earlier and can be reused. We only need to sign the bob_sweeper input using key path spending.

In [ ]:
# SIGHASH for key path spend
sighash_epoch = bytes.fromhex("00")
hash_type_keypath = bytes.fromhex("00")  # SIGHASH_DEFAULT (SIGHASH_ALL)

# TODO: Transaction data - now includes both inputs
bob_sweeper_value = # TODO
bob_sweeper_value_bytes = bob_sweeper_value.to_bytes(8, byteorder="little", signed=False)
sha_prevouts_v2 = # TODO
sha_amounts_v2 = # TODO

# TODO: scriptPubKeys for both inputs
bob_accepted_htlc_spk_with_len = # TODO
bob_sweeper_spk = # TODO
bob_sweeper_spk_with_len = # TODO
sha_scriptpubkeys_v2 = # TODO

sha_sequences_v2 = sha256(sequence_htlc + sequence_change).digest()
sha_outputs_v2 = sha256(outputs_v2).digest()

# Data about this input (key path spend)
spend_type = bytes.fromhex("00")  # key path, no annex

# Index of the input being signed (input 1, the sweeper input)
input_index = bytes.fromhex("01000000")

sig_msg_sweeper = (
    sighash_epoch
    + hash_type_keypath
    + version
    + locktime
    + sha_prevouts_v2
    + sha_amounts_v2
    + sha_scriptpubkeys_v2
    + sha_sequences_v2
    + sha_outputs_v2
    + spend_type
    + input_index
)

tag_hash = sha256("TapSighash".encode()).digest()
sighash_sweeper = sha256(tag_hash + tag_hash + sig_msg_sweeper).digest()

# TODO: Sign with bob_sweeper_privkey
import secrets
aux_sweeper = secrets.token_bytes(32)
bob_sweeper_sig = # TODO

# Verify the signature
bob_sweeper_sig_valid = bob_sweeper_pubkey.verify_schnorr(bob_sweeper_sig, sighash_sweeper)
print("Bob sweeper signature valid?", bob_sweeper_sig_valid)

### Build the final signed transaction

Now we construct the witness data for both inputs and build the final signed transaction.

In [ ]:
# TODO: Witness for Input 1 (bob sweeper) - key path spend
witness_input1 = # TODO

# Complete witness data (HTLC witness + sweeper witness)
witness_v2 = witness + witness_input1

# The final signed transaction
signed_tx_v2 = (
    version
    + marker
    + flag
    + input_count_v2
    + inputs_v2
    + output_count_v2
    + outputs_v2
    + witness_v2
    + locktime
)

print("signed tx v2:", signed_tx_v2.hex())

# Decode the signed transaction
decoded_signed_v2 = node.decoderawtransaction(signed_tx_v2.hex())
print("\n" + json.dumps(decoded_signed_v2, indent=2, default=str))

In [ ]:
# Test mempool accept
print("\nTest mempool accept:")
try:
    result = node.testmempoolaccept(rawtxs=[signed_tx_v2.hex()])
    print(json.dumps(result, indent=2, default=str))
    
    if result[0]['allowed']:
        print("\n✓ Transaction is valid and meets minimum relay fee!")
        print(f"  Fee: {result[0].get('fees', {}).get('base', 'N/A')} BTC")
    else:
        print(f"\n✗ Transaction rejected: {result[0].get('reject-reason', 'Unknown')}")
except Exception as e:
    print(f"Error: {e}")

## Summary

In this chapter, we successfully created an HTLC-success transaction that spends the accepted HTLC output from Bob's commitment transaction. Key points:

1. **Purpose**: The HTLC-success transaction allows Bob to claim funds from an accepted HTLC by providing the payment preimage.

2. **Script Path Spending**: Uses script path spending through the Taproot tree, specifically scriptB (success path).

3. **Payment Preimage**: The critical component - Bob must provide the preimage that hashes to the payment_hash.

4. **Two Signatures Required**: The success script requires signatures from both Bob's and Alice's HTLC keys.

5. **Delayed Output**: The output is similar to `to_local` - spendable by Bob after a delay (to_self_delay), or immediately by Alice if she has the revocation key.

6. **Witness Structure**: The witness includes both signatures, the payment preimage, the script, and a control block proving the script is part of the Taproot tree.

7. **Zero Fee**: Like HTLC-timeout, this transaction has zero fee and must be combined with additional inputs to pay for mining fees.

8. **Key Difference from HTLC-Timeout**: 
   - HTLC-timeout: Alice reclaims after timeout (scriptA)
   - HTLC-success: Bob claims with preimage (scriptB)